---
title: "PostgreSQL in Docker"
format:
    pdf:
        toc: true
        number-section: true
        colorlinks: true
jupyter: python3
---

## Configs

In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT # <-- ADD THIS LINE

## Docker startup

In [4]:
# ensure Docker is running before executing these commands
!docker run --name pg_local -p 5432:5432 -e POSTGRES_USER=sde -e POSTGRES_PASSWORD=password -e POSTGRES_DB=scd2 -d postgres:12.2

eb3d284065f3c947b32f8b91a1f13f4255ff8909f49e6896163dabe1bea73e14


In [5]:
# password is password
!pgcli -h localhost -p 5432 -U sde scd2

'pgcli' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
# make sure you have docker running
# and your postgresql container running
!docker ps

CONTAINER ID   IMAGE           COMMAND                  CREATED          STATUS          PORTS                    NAMES
eb3d284065f3   postgres:12.2   "docker-entrypoint.s…"   12 seconds ago   Up 11 seconds   0.0.0.0:5432->5432/tcp   pg_local


## PostgreSQL config

In [7]:
dbname='scd2'
user = 'sde'
host='localhost:5432'
password = 'password'

In [8]:
connection = psycopg2.connect(f"dbname={dbname} user={user} password={password}")

In [9]:
connection.autocommit = True

In [10]:
cur = connection.cursor()

In [12]:
db_version = cur.fetchone()

In [13]:
print(db_version)

('PostgreSQL 12.2 (Debian 12.2-2.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',)


## Test SQL commands

In [53]:
# create test database
command = """
CREATE DATABASE warehouse;
"""

In [54]:
cur.execute(command)

In [16]:
# create user table and fill it with values
user_table_command = """
DROP TABLE IF EXISTS user_dim;
CREATE TABLE user_dim (
    user_key BIGINT,
    user_id VARCHAR(40),
    first_name VARCHAR(10),
    last_name VARCHAR(10),
    address VARCHAR(100),
    zipcode VARCHAR(10),
    created_datetime TIMESTAMP,
    updated_datetime TIMESTAMP,
    row_effective_datetime TIMESTAMP,
    row_expiration_datetime TIMESTAMP,
    current_row_indicator VARCHAR(10)
);
INSERT INTO user_dim (
        user_key,
        user_id,
        first_name,
        last_name,
        address,
        zipcode,
        created_datetime,
        updated_datetime,
        row_effective_datetime,
        row_expiration_datetime,
        current_row_indicator
    )
VALUES (
        1000,
        'b0cc9fde-a29a-498e-824f-e52399991beb',
        'john',
        'doe',
        'world',
        10027,
        '2020-01-01 10:00:00',
        '2020-01-01 10:00:00',
        ' 2020-01-01 10:00:00',
        '2021-01-01 17:59:59',
        'expired'
    ),
    (
        1200,
        'b0cc9fde-a29a-498e-824f-e52399991beb',
        'john',
        'doe',
        'world',
        10012,
        '2020-01-01 10:00:00',
        '2021-01-01 18:00:00',
        '2021-01-01 18:00:00',
        '9999-12-31 00:00:00',
        'current'
    );
"""

In [17]:
cur.execute(user_table_command)

## Confirm our work

In [18]:
# now we check our work
select_user_table = """
select *
from user_dim
"""

In [19]:
cur.execute(select_user_table)

In [20]:
records = cur.fetchall()

In [21]:
for row in records:
    print(row)

(1000, 'b0cc9fde-a29a-498e-824f-e52399991beb', 'john', 'doe', 'world', '10027', datetime.datetime(2020, 1, 1, 10, 0), datetime.datetime(2020, 1, 1, 10, 0), datetime.datetime(2020, 1, 1, 10, 0), datetime.datetime(2021, 1, 1, 17, 59, 59), 'expired')
(1200, 'b0cc9fde-a29a-498e-824f-e52399991beb', 'john', 'doe', 'world', '10012', datetime.datetime(2020, 1, 1, 10, 0), datetime.datetime(2021, 1, 1, 18, 0), datetime.datetime(2021, 1, 1, 18, 0), datetime.datetime(9999, 12, 31, 0, 0), 'current')
